In [1]:
import pandas as pd
import numpy as np
import os
import cv2
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import VGG16
from tensorflow.keras import models, layers
from tensorflow.keras import layers, optimizers 
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.regularizers import l2


2023-12-14 13:08:46.326669: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512_VNNI
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-14 13:08:46.519043: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
# Import data
class_names_dict = np.load("class_names.npy", allow_pickle=True).item()
attributes_npy = np.load("attributes.npy", allow_pickle=True)
attributes_df = pd.read_csv("attributes.txt", header=None, index_col = 0, sep=" |::", names=("attribute", "value"), engine='python')
training_labels_df = pd.read_csv("train_images.csv", header = 0)
test_images_paths = pd.read_csv("test_images_path.csv", header = 0)
test_images_sample = pd.read_csv("test_images_sample.csv", header = 0)

In [3]:
# local paths 
training_dir = "train_images_inferred_bad5/train_images" #This is the inferred augmented dir
testing_dir = "test_images" 

In [4]:
# This loads the training images from the balanced augmented training dir

images = []
labels = []

resize_dim = (224, 224)

# Iterate through subdirectories
for subdir in range(1, 201):
    subdir_path = os.path.join(training_dir, str(subdir))

    # Iterate through files in each subdirectory
    for filename in os.listdir(subdir_path):
        if filename.endswith(".jpg"):
            
            # Read images
            image_path = os.path.join(subdir_path, filename)
            img = cv2.imread(image_path)
            
            # Fix images
            img_resized = cv2.resize(img, resize_dim)
            img_normalized = img_resized / 255.0
            
            #Load images
            images.append(img_normalized)
            labels.append(subdir)

# Convert lists to numpy arrays
images_array = np.array(images)
labels_array = np.array(labels)


KeyboardInterrupt: 

In [14]:
# Split to training and val set

train_birds_set, val_birds_set, train_labels_set, val_labels_set = train_test_split(images, labels, test_size=0.2, random_state=42)

In [15]:
# Convert sets to arrays because models dont accept lists

train_birds_set = np.array(train_birds_set)
val_birds_set = np.array(val_birds_set)
train_labels_set = np.array(train_labels_set)
val_labels_set = np.array(val_labels_set)

In [60]:

base_model = tf.keras.applications.InceptionV3(include_top= False,)

base_model.trainable = False
inputs = tf.keras.layers.Input(shape =(224,224,3),)


x = base_model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(201, activation = "softmax", name = "output-layer")(x)
model = tf.keras.Model(inputs, outputs)


base_model.trainable = True

for layer in base_model.layers[:-40]:
    layer.trainable = False

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


model.summary()

Shape after passing inputs through base model: (None, 5, 5, 2048)
Shape after GlobalAveragePooling2D: (None, 2048)
Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input-layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 inception_v3 (Functional)   (None, None, None, 2048)  21802784  
                                                                 
 global_average_pooling_laye  (None, 2048)             0         
 r (GlobalAveragePooling2D)                                      
                                                                 
 output-layer (Dense)        (None, 201)               411849    
                                                                 
Total params: 22,214,633
Trainable params: 6,485,577
Non-trainable params: 15,729,056
________________________________________________________

In [61]:

datagen = ImageDataGenerator(

    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    vertical_flip=True,
)


datagen.fit(train_birds_set)

In [62]:
history = model.fit(train_birds_set, train_labels_set, epochs=10, 
                    validation_data=(val_birds_set, val_labels_set))

Epoch 1/10


/sw/arch/RHEL8/EB_production/2022/software/TensorFlow/2.11.0-foss-2022a-CUDA-11.7.0/lib/python3.10/site-packages/keras/backend.py:5585: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


250/250 [==============================] - 16s 30ms/step - loss: 2.3106 - accuracy: 0.4884 - val_loss: 1.1159 - val_accuracy: 0.7020
Epoch 2/10
250/250 [==============================] - 5s 19ms/step - loss: 0.3917 - accuracy: 0.9054 - val_loss: 0.8046 - val_accuracy: 0.7800
Epoch 3/10
250/250 [==============================] - 5s 19ms/step - loss: 0.1024 - accuracy: 0.9793 - val_loss: 0.5620 - val_accuracy: 0.8505
Epoch 4/10
250/250 [==============================] - 5s 19ms/step - loss: 0.0225 - accuracy: 0.9973 - val_loss: 0.4771 - val_accuracy: 0.8830
Epoch 5/10
 37/250 [===>..........................] - ETA: 3s - loss: 0.0065 - accuracy: 1.0000

KeyboardInterrupt: 

In [70]:
#model_0.save('best_model_49.h5')

## Test

In [63]:
## Read images from test_dir (resize and rescale) in order and load test_images[] array


resize_dim = (224, 224)

def read_image(image_path):
    image = cv2.imread(testing_dir+image_path)
    img_resized = cv2.resize(image, resize_dim)
    img_normalized = img_resized / 255.0
    return img_normalized


test_images_paths['pixel_array'] = test_images_paths['image_path'].apply(read_image)

test_images = np.array(test_images_paths['pixel_array'].tolist())

In [64]:
predictions = model_0.predict(test_images)

125/125 [==============================] - 3s 12ms/step


In [65]:
predicted_labels = np.argmax(predictions, axis=1)

In [66]:
predicted_labels

array([ 67,  37,  74, ..., 135,  62, 111])

In [67]:
test_images_sample['label'] = predicted_labels

In [68]:
test_images_sample

,id,label
0,1,67
1,2,37
2,3,74
3,4,12
4,5,74
...,...,...
3995,3996,21
3996,3997,55
3997,3998,135
3998,3999,62


In [69]:
csv_file_path = 'kaggle_file.csv'

# Save the DataFrame to CSV
test_images_sample.to_csv(csv_file_path, index=False)

# Display a message indicating the file has been saved
print(f'DataFrame has been saved to {csv_file_path}')

DataFrame has been saved to kaggle_file.csv
